# MNIST Demo 1 - Deep Learning as a Service
## 1. Prepare Environment

In [ ]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
wml_credentials = { "url"         : "https://ibm-watson-ml.mybluemix.net",
                    "username"    : "***",
                    "password"    : "***",
                    "instance_id" : "***"
                   }
client = WatsonMachineLearningAPIClient( wml_credentials )

## 2. Prepare Training Definition

In [ ]:
!pip install --upgrade wget

In [ ]:
import os
import wget

filename = 'tf-model.zip'
url = 'https://github.com/pmservice/wml-sample-models/blob/master/tensorflow/hand-written-digit-recognition/definition/tf-model.zip?raw=true'

if not os.path.isfile( filename ): 
    wget.download( url )

In [ ]:
!ls -l

In [ ]:
metadata = {
client.repository.DefinitionMetaNames.NAME              : "python-client-tutorial_training-definition",
client.repository.DefinitionMetaNames.FRAMEWORK_NAME    : "tensorflow",
client.repository.DefinitionMetaNames.FRAMEWORK_VERSION : "1.5",
client.repository.DefinitionMetaNames.RUNTIME_NAME      : "python",
client.repository.DefinitionMetaNames.RUNTIME_VERSION   : "3.5",
client.repository.DefinitionMetaNames.EXECUTION_COMMAND : "python3 convolutional_network.py --trainImagesFile ${DATA_DIR}/train-images-idx3-ubyte.gz --trainLabelsFile ${DATA_DIR}/train-labels-idx1-ubyte.gz --testImagesFile ${DATA_DIR}/t10k-images-idx3-ubyte.gz --testLabelsFile ${DATA_DIR}/t10k-labels-idx1-ubyte.gz --learningRate 0.001 --trainingIters 20000"
}
definition_details = client.repository.store_definition( filename, meta_props=metadata )
definition_uid     = client.repository.get_definition_uid( definition_details )
print( "definition_uid: ", definition_uid )

## 3. Train DL model in WML
### 3.1 Submit Training Job

In [ ]:
metadata = {
client.training.ConfigurationMetaNames.NAME         : "python-client-tutorial_training-run",
client.training.ConfigurationMetaNames.TRAINING_DATA_REFERENCE : {
   "connection" : { 
      "endpoint_url"      : "***",
      "access_key_id"     : "***",
      "secret_access_key" : "***"
      },
   "source" : { 
      "bucket" : "***",
      },
      "type" : "s3"
   },
client.training.ConfigurationMetaNames.TRAINING_RESULTS_REFERENCE: {
   "connection" : {
      "endpoint_url"      : "***",
      "access_key_id"     : "***",
      "secret_access_key" : "***"
      },
      "target" : {
         "bucket" : "***",
      },
      "type" : "s3"
   }
}

In [ ]:
run_details = client.training.run( definition_uid, meta_props=metadata )
run_uid     = client.training.get_run_uid( run_details )
print( "run_uid: ", run_uid )

### 3.2 Monitor Training Status

In [ ]:
client.training.get_status( run_uid )

In [ ]:
import time

time_s = time.time()

res = {'state': ''}
while res['state'] != 'completed':
    time.sleep(5)  
    res = client.training.get_status( run_uid )
    
print('Model training took %s seconds' % (time.time() - time_s))

## 4. Store and Deploy Model

In [ ]:
stored_model_name    = "python-client-tutorial_model"
stored_model_details = client.repository.store_model( run_uid, stored_model_name )
model_uid            = client.repository.get_model_uid( stored_model_details )
print( "model_uid: ", model_uid )

In [ ]:
deploymnt_name   = "MNIST-handwriting"
deployment_desc  = "Online deployment of Python client tutorial model"
deployment       = client.deployments.create( model_uid, deploymnt_name, deployment_desc )
scoring_endpoint = client.deployments.get_scoring_url( deployment )
print( "scoring_endpoint: ", scoring_endpoint )

## 5. Score Incoming Data
### 5.1 Prepare Test Data

In [ ]:
filename = 'tf-mnist-test-payload.json'
url = 'https://raw.githubusercontent.com/pmservice/wml-sample-models/master/tensorflow/hand-written-digit-recognition/test-data/tf-mnist-test-payload.json'

if not os.path.isfile( filename ): 
    wget.download( url )

In [ ]:
import json

with open( filename ) as data_file: 
    test_data = json.load( data_file )
    
payload = test_data[ 'payload' ]

### 5.2 Invoke Scoring Service

In [ ]:
client.deployments.score( scoring_endpoint, payload )

### 5.3 Visualize Ground Truth

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

with open(filename) as f:
    data = json.load(f)

In [ ]:
values = data['payload']['values']

for i in range(len(values)):
    pixels = np.array(values[i])
    pixels = pixels.reshape((28, 28))

    plt.imshow(pixels, cmap='gray')
    plt.show()